In [1]:
import datetime
import glob
import os
import os.path as osp
import platform
import pprint
import random
import shlex
import shutil
import signal
import subprocess
import sys
import threading

import numpy as np
import torch
import yaml
from docopt import docopt
import scipy.io as sio
import lcnn
from lcnn.config import C, M
from lcnn.datasets import WireframeDataset, collate
from lcnn.models.line_vectorizer import LineVectorizer
from lcnn.models.multitask_learner import MultitaskHead, MultitaskLearner
from lcnn.models.HT import hough_transform

In [4]:
from torch.utils.data import TensorDataset, DataLoader
class SimpleCustomBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        self.inp = torch.stack(transposed_data[0], 0)
        self.tgt = torch.stack(transposed_data[1], 0)

    # custom memory pinning method on custom type
    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.tgt = self.tgt.pin_memory()
        return self

def collate_wrapper(batch):
    return SimpleCustomBatch(batch)

inps = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
tgts = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
dataset = TensorDataset(inps, tgts)

loader = DataLoader(dataset, batch_size=2, collate_fn=collate_wrapper,
                    pin_memory=True)

for batch_ndx, sample in enumerate(loader):
    print(sample.inp.is_pinned())
    print(sample.tgt.is_pinned())

True
True
True
True
True
True
True
True
True
True


In [5]:
with open('./myTmp.ipynb', 'r') as f:
    identifier = id(f)
    print(identifier)

1276189738688


In [7]:
rootdir = './figs'
filelist = glob.glob(f"{rootdir}/exp_*.png")

In [11]:
from skimage import io
iname = filelist[1]
image = io.imread(iname).astype(float)[:, :, :3]

In [19]:
image.shape

(555, 555, 3)

In [23]:
image[:, ::-1, :].shape

(555, 555, 3)